In [2]:
import pandas as pd
import os
import numpy as np

In [3]:
# in pub pcr events, e situation 13 = initial patient acuity, e disposition 23 = hospital capability

In [4]:
folder = r'C:\Users\gratt\Desktop\emsData\processedCSVs\big\pub_pcrevents_chunks'

In [5]:
filepaths = [os.path.join(folder, filename) for filename in os.listdir(folder)]
filepaths

['C:\\Users\\gratt\\Desktop\\emsData\\processedCSVs\\big\\pub_pcrevents_chunks\\pub_pcrevents_chunk1.csv',
 'C:\\Users\\gratt\\Desktop\\emsData\\processedCSVs\\big\\pub_pcrevents_chunks\\pub_pcrevents_chunk10.csv',
 'C:\\Users\\gratt\\Desktop\\emsData\\processedCSVs\\big\\pub_pcrevents_chunks\\pub_pcrevents_chunk2.csv',
 'C:\\Users\\gratt\\Desktop\\emsData\\processedCSVs\\big\\pub_pcrevents_chunks\\pub_pcrevents_chunk3.csv',
 'C:\\Users\\gratt\\Desktop\\emsData\\processedCSVs\\big\\pub_pcrevents_chunks\\pub_pcrevents_chunk4.csv',
 'C:\\Users\\gratt\\Desktop\\emsData\\processedCSVs\\big\\pub_pcrevents_chunks\\pub_pcrevents_chunk5.csv',
 'C:\\Users\\gratt\\Desktop\\emsData\\processedCSVs\\big\\pub_pcrevents_chunks\\pub_pcrevents_chunk6.csv',
 'C:\\Users\\gratt\\Desktop\\emsData\\processedCSVs\\big\\pub_pcrevents_chunks\\pub_pcrevents_chunk7.csv',
 'C:\\Users\\gratt\\Desktop\\emsData\\processedCSVs\\big\\pub_pcrevents_chunks\\pub_pcrevents_chunk8.csv',
 'C:\\Users\\gratt\\Desktop\\emsData

In [6]:
def importvars(filepath):
    return pd.read_csv(filepath, usecols=["PcrKey", "eSituation_02", "eSituation_13", "eDisposition_23"])

In [7]:
df = pd.concat(map(importvars, filepaths))

C:\Users\gratt\AppData\Local\Temp\ipykernel_13184\3920843801.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(filepath, usecols=["PcrKey", "eSituation_02", "eSituation_13", "eDisposition_23"])
C:\Users\gratt\AppData\Local\Temp\ipykernel_13184\3920843801.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(filepath, usecols=["PcrKey", "eSituation_02", "eSituation_13", "eDisposition_23"])
C:\Users\gratt\AppData\Local\Temp\ipykernel_13184\3920843801.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(filepath, usecols=["PcrKey", "eSituation_02", "eSituation_13", "eDisposition_23"])
C:\Users\gratt\AppData\Local\Temp\ipykernel_13184\3920843801.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read

In [8]:
df

,PcrKey,eDisposition_23,eSituation_02,eSituation_13
0,1472192,7701003,7701003,2813005
1,13273412,9908007,9922001,2813005
2,14548133,7701003,7701001,2813001
3,17184298,7701003,9922005,2813001
4,21699657,7701003,9922001,2813005
...,...,...,...,...
4999995,168637681,7701001,7701003,7701001
4999996,168637682,9908007,9922001,7701003
4999997,168637683,7701003,9922001,2813005
4999998,168637684,9908007,9922001,2813005


In [9]:
# filter for only patients where possible injury = yes
df = df[ (df['eSituation_02'] == 9922005)]

In [10]:
df['eSituation_02'].value_counts(normalize=True)

9922005    1.0
Name: eSituation_02, dtype: float64

In [11]:
# drop the injury column
df = df.drop(columns="eSituation_02")

In [12]:
# eSituation_13 == initial patient acuity
# eDisposition_23 = hospital capability
df = df.rename(columns = {'eDisposition_23':'hospCapability', 'eSituation_13':'initialPtAcuity'})

In [13]:
df

,PcrKey,hospCapability,initialPtAcuity
3,17184298,7701003,2813001
14,102488422,7701003,2813001
17,113861204,7701003,2813003
18,116400273,7701003,2813003
27,118170842,7701001,7701003
...,...,...,...
4999924,168637610,7701003,7701003
4999943,168637629,7701003,2813005
4999954,168637640,7701003,2813005
4999974,168637660,9908007,7701003


In [14]:
df['initialPtAcuity'].value_counts(normalize=True)

2813005    0.521749
7701003    0.234076
2813003    0.181332
2813001    0.036845
7701001    0.021813
2813007    0.004186
Name: initialPtAcuity, dtype: float64

initial patient acuity code list
2813001 - Critical (Red)
2813003 - Emergent (Yellow)
2813005 - Lower Acuity (Green)
2813007 - Dead without Resuscitation Efforts (Black) 
7701001 - NA
7701003 - not recorded

In [15]:
# new codes
# critical == 4
df = df.replace([2813001], 4)
# emergent == 3
df = df.replace([2813003], 3)
# lower acuity == 3
df = df.replace([2813005], 2)
# dead = 1
df = df.replace([2813007], 1)

In [16]:
# throw out missing data
df = df[ (df['initialPtAcuity'] == 1) | (df['initialPtAcuity'] == 2) | (df['initialPtAcuity'] == 3) | (df['initialPtAcuity'] == 4)]

In [17]:
df

,PcrKey,hospCapability,initialPtAcuity
3,17184298,7701003,4
14,102488422,7701003,4
17,113861204,7701003,3
18,116400273,7701003,3
32,118170851,9908007,2
...,...,...,...
4999920,168637606,7701003,2
4999923,168637609,9908007,3
4999943,168637629,7701003,2
4999954,168637640,7701003,2


In [18]:
df['initialPtAcuity'].value_counts(normalize=True)

2    0.701170
3    0.243690
4    0.049515
1    0.005625
Name: initialPtAcuity, dtype: float64

In [19]:
# throw out missing data for hosp capability
df = df[ (df['hospCapability'] != 7701003) & (df['hospCapability'] != 7701001)]

In [20]:
df

,PcrKey,hospCapability,initialPtAcuity
32,118170851,9908007,2
33,118170860,9908007,3
70,118170980,9908007,3
134,118171192,9908007,3
140,118171212,9908007,2
...,...,...,...
4999748,168637433,9908007,2
4999767,168637452,9908007,3
4999905,168637590,9908007,3
4999923,168637609,9908007,3


In [21]:
df['hospCapability'].value_counts(normalize=True)

9908007    0.701246
9908007    0.055648
9908021    0.052812
7701003    0.049990
9908023    0.042551
7701001    0.027554
9908005    0.015022
9908025    0.014860
9908011    0.008993
9908027    0.004882
9908021    0.004304
9908023    0.003896
9908033    0.002970
9908017    0.002766
9908003    0.002099
9908025    0.001996
9908001    0.001804
9908005    0.001255
9908027    0.001078
9908031    0.000948
9908011    0.000886
9908019    0.000643
9908009    0.000425
9908029    0.000280
9908017    0.000248
9908033    0.000179
9908003    0.000168
9908001    0.000130
9908035    0.000121
9908031    0.000106
9908019    0.000064
9908029    0.000031
9908009    0.000029
Unknown    0.000013
9908035    0.000002
Name: hospCapability, dtype: float64

In [22]:
# idk how that unknown got in there but delete
df = df[ df['hospCapability'] != "Unknown" ]

In [23]:
df['hospCapability'].value_counts(normalize=True)

9908007    0.701255
9908007    0.055649
9908021    0.052813
7701003    0.049991
9908023    0.042552
7701001    0.027554
9908005    0.015022
9908025    0.014860
9908011    0.008993
9908027    0.004882
9908021    0.004304
9908023    0.003896
9908033    0.002970
9908017    0.002766
9908003    0.002099
9908025    0.001996
9908001    0.001804
9908005    0.001255
9908027    0.001078
9908031    0.000948
9908011    0.000886
9908019    0.000643
9908009    0.000425
9908029    0.000280
9908017    0.000248
9908033    0.000179
9908003    0.000168
9908001    0.000130
9908035    0.000121
9908031    0.000106
9908019    0.000064
9908029    0.000031
9908009    0.000029
9908035    0.000002
Name: hospCapability, dtype: float64

In [24]:
df = df.astype(int)

In [25]:
# good now everything is an integer
df['hospCapability'].value_counts(normalize=True)

9908007    0.756904
9908021    0.057117
7701003    0.049991
9908023    0.046448
7701001    0.027554
9908025    0.016856
9908005    0.016277
9908011    0.009879
9908027    0.005959
9908033    0.003149
9908017    0.003015
9908003    0.002267
9908001    0.001934
9908031    0.001055
9908019    0.000707
9908009    0.000455
9908029    0.000310
9908035    0.000123
Name: hospCapability, dtype: float64

In [26]:
# codes for trauma centers
tmaCnts = [9908021, 9908023, 9908025, 9908027, 9908029]
# codes for non trauma centers
notTma = [9908001, 9908003, 9908005, 9908007, 9908009, 9908011, 9908017, 9908019, 9908031, 9908033, 9908035]

In [27]:
# replace all vals for non trauma centers with zeroes
df = df.replace(notTma, value=[0]*11)
# replace all vals for trauma centers with ones
df = df.replace(tmaCnts, value=[1]*5)

In [28]:
df.head(25)

,PcrKey,hospCapability,initialPtAcuity
32,118170851,0,2
33,118170860,0,3
70,118170980,0,3
134,118171192,0,3
140,118171212,0,2
145,118171228,0,3
165,118171283,0,2
177,118171324,0,4
179,118171328,0,2
182,118171353,0,3


In [29]:
df['hospCapability'].value_counts(normalize=True)

0          0.795764
1          0.126691
7701003    0.049991
7701001    0.027554
Name: hospCapability, dtype: float64

In [30]:
# throw out missing data for hosp capability
df = df[ (df['hospCapability'] != 7701003) & (df['hospCapability'] != 7701001)]

In [31]:
df['hospCapability'].value_counts(normalize=True)

0    0.862659
1    0.137341
Name: hospCapability, dtype: float64

In [32]:
# now join with socioeoncomic
raceAgeSex = pd.read_csv(r"C:\Users\gratt\Desktop\emsData\raceAgeSex.csv")

In [33]:
raceAgeSex

,PcrKey,white,black,hispanic_latino,asian,americanIndian_alaskaNative,nativeHawaiian_otherPacificIslander,sex,age
0,13273412,1,0,0,0,0,0,0,78
1,17184298,1,0,0,0,0,0,0,10
2,21699657,1,0,0,0,0,0,0,84
3,72057010,1,0,0,0,0,0,0,84
4,83733589,0,0,0,0,1,0,0,31
...,...,...,...,...,...,...,...,...,...
32514131,182540290,0,1,0,0,0,0,0,77
32514132,182540377,0,1,0,0,0,0,1,54
32514133,182540383,0,1,0,0,0,0,0,78
32514134,182541657,1,0,0,0,0,0,1,74


In [34]:
df = df.set_index('PcrKey')

In [35]:
df

,hospCapability,initialPtAcuity
PcrKey,,
118170851,0,2
118170860,0,3
118170980,0,3
118171192,0,3
118171212,0,2
...,...,...
168637433,0,2
168637452,0,3
168637590,0,3


In [36]:
raceAgeSex = raceAgeSex.set_index('PcrKey')

In [37]:
raceAgeSex

,white,black,hispanic_latino,asian,americanIndian_alaskaNative,nativeHawaiian_otherPacificIslander,sex,age
PcrKey,,,,,,,,
13273412,1,0,0,0,0,0,0,78
17184298,1,0,0,0,0,0,0,10
21699657,1,0,0,0,0,0,0,84
72057010,1,0,0,0,0,0,0,84
83733589,0,0,0,0,1,0,0,31
...,...,...,...,...,...,...,...,...
182540290,0,1,0,0,0,0,0,77
182540377,0,1,0,0,0,0,1,54
182540383,0,1,0,0,0,0,0,78


In [38]:
df = df.join(raceAgeSex)

In [39]:
df

,hospCapability,initialPtAcuity,white,black,hispanic_latino,asian,americanIndian_alaskaNative,nativeHawaiian_otherPacificIslander,sex,age
PcrKey,,,,,,,,,,
118170851,0,2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,16.0
118170860,0,3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,49.0
118170980,0,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0
118171192,0,3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,58.0
118171212,0,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,41.0
...,...,...,...,...,...,...,...,...,...,...
168637433,0,2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,45.0
168637452,0,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0
168637590,0,3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,51.0


In [40]:
df = df.dropna()

In [41]:
df

,hospCapability,initialPtAcuity,white,black,hispanic_latino,asian,americanIndian_alaskaNative,nativeHawaiian_otherPacificIslander,sex,age
PcrKey,,,,,,,,,,
118170851,0,2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,16.0
118170860,0,3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,49.0
118170980,0,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0
118171192,0,3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,58.0
118171212,0,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,41.0
...,...,...,...,...,...,...,...,...,...,...
168637433,0,2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,45.0
168637452,0,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0
168637590,0,3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,51.0


In [42]:
df = df.astype(int)

In [43]:
df

,hospCapability,initialPtAcuity,white,black,hispanic_latino,asian,americanIndian_alaskaNative,nativeHawaiian_otherPacificIslander,sex,age
PcrKey,,,,,,,,,,
118170851,0,2,0,0,1,0,0,0,1,16
118170860,0,3,1,0,0,0,0,0,1,49
118170980,0,3,1,0,0,0,0,0,0,30
118171192,0,3,0,1,0,0,0,0,1,58
118171212,0,2,1,0,0,0,0,0,0,41
...,...,...,...,...,...,...,...,...,...,...
168637433,0,2,1,0,0,0,0,0,1,45
168637452,0,3,1,0,0,0,0,0,0,15
168637590,0,3,1,0,0,0,0,0,1,51


In [44]:
# looks good export
df.to_csv(r"C:\Users\gratt\Desktop\emsData\raceAgeSexAcuityTrauma.csv")